# DB2-Salesforce populate survey URLs obtain from DB2

In [1]:
# Parameters
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"shit9927289sSYTkLiGvyK2UrazHFmjFUGU",
} 

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')

import pandas as pd
import datetime

## Get the URL code from DB2


In [3]:
# connect with DB2
import sqlalchemy as sql

engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub' \
                                               %(sql_login_params['username'], sql_login_params['password']))

# page_id indicates the question
url_code_df = pd.read_sql_query("select user_id, code from jos_reply_access_codes \
where page_id = 2 order by user_id desc limit 5000;",engine)


In [4]:
display(url_code_df.head(2))

,user_id,code
0,998,rWB03lnQxP7z2r1pE0LBIIC9QwgNDGkFMSH3gSTpnMtgoO...
1,1000,ejjJycB4jtnhde8xQDO0t1zXFtvpzsQxSG9Sprku66EjYv...


In [5]:
## populating full url codes
url_base = "https://nanohub.org/reply/pages/2?code="

from copy import deepcopy
codes = deepcopy(url_code_df['code'])

#import time
#st = time.time()
codes = codes.apply(lambda x: url_base+x)
#tt = st-time.time()
#print(tt)
print(codes[0])


https://nanohub.org/reply/pages/2?code=rWB03lnQxP7z2r1pE0LBIIC9QwgNDGkFMSH3gSTpnMtgoO8t8EBf8dLN9pFKjXj7


In [6]:
## add the new code to the dataframe
url_code_df['code'] = codes
url_code_df = url_code_df.rename(columns={"user_id": "nanoHUB_user_ID__c", "code":"nh_use_survey_link__c"})
display(url_code_df.head(2))

,nanoHUB_user_ID__c,nh_use_survey_link__c
0,998,https://nanohub.org/reply/pages/2?code=rWB03ln...
1,1000,https://nanohub.org/reply/pages/2?code=ejjJycB...


## Send to SF

In [9]:
from DB2SalesforceAPI import DB2SalesforceAPI

# create DB2 to Salesforce API object
db_s = DB2SalesforceAPI(sf_login_params)


Obtained Salesforce access token ...... True


In [10]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(url_code_df)

[Success] Bulk job creation successful. Job ID = 7505w00000PTdsFAAT
hello
[Success] CSV upload successful. Job ID = 7505w00000PTdsFAAT
[Success] Closing job successful. Job ID = 7505w00000PTdsFAAT


In [23]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000PTdsFAAT',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2020-08-26T19:32:07.000+0000',
 'systemModstamp': '2020-08-26T19:41:29.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 216919,
 'numberRecordsFailed': 18,
 'retries': 0,
 'totalProcessingTime': 2244470,
 'apiActiveProcessingTime': 2049434,
 'apexProcessingTime': 21749601}

In [24]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",nanoHUB_user_ID__c,nh_use_survey_link__c\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","62873.0","https://nanohub.org/reply/pages/2?code=uw6ThU98ORPE1ZwqgtQQafU2ZCpifupapFYwobnNDFsdO4wlLDM7NJYft7OxdRWt"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","185228.0","https://nanohub.org/reply/pages/2?code=9aY0c9CvjqYB4S8c82gNXs2EQJl9RllFwCeskuTHytejoV0fe8DTCxEUdNxL2hLC"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","1000.0","https://nanohub.org/reply/pages/2?code=ejjJycB4jtnhde8xQDO0t1zXFtvpzsQxSG9Sprku66EjYvMVpwaXy3udNb2eEw6C"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","1819.0","https://nanohub.org/reply/pages/2?code=gadZ6YRCzsTnrBxChNlIn2taLjL5l2RbHl2Vtn6MJXL3r3V6WZbniuVjI5SPCKOM"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","4311.0","https://nanohub.or